In [40]:
import numpy as np  
from itertools import combinations
import pandas as pd

# CARGA DE DATOS

In [41]:
spotify = np.load("spotify.npy",allow_pickle=True)
spotify = spotify.tolist()

# FUNCIONES ADICIONALES

In [48]:
### HALLA LA FRECUENCIA (APARICIONES) DE CADA CANCION EN LAS PLAYLISTS ##
def frecuency_each_song( spotify ):
    
    frecuencias = {}
    # RECORREMOS LAS PLAYLIST Y GUARGAMOS LA FRECUENCIA EN UN DICCIONARIO
    for i in range( len(spotify) ):
        for j in range(len(spotify[i])):
            song = spotify[i][j]
            if song in frecuencias: frecuencias[song] += 1
            else: frecuencias[song] = 1
    return frecuencias

### HALLA LA FRECUENCIA (APARICIONES) DE UN ARREGLO DE CANCIONES EN LAS PLAYLISTS ##
def frecuency_each_combination( spotify, combination, n ):
    # RECORRER ARREGLO DE PLAYLISTS
    for i in range( len(spotify) ):
        # RECORRER DICCIONARIO QUE CONTENGA LAS COMBINACIONES
        for key, value in combination.items():
            playlist = []
            for j in range(n): playlist.append(key[j])
            # VERIFICAR SI COMBINACION SE ENCUENTRA EN PLAYLIST
            if( set(playlist).issubset(set( spotify[i] ))):  
                combination[key] += 1
    return combination

### REALIZA LAS COMBINACIONES DE "N" CANCIONES Y LOS GUARDA EN UN DICCIONARIO ##
def create_combinations( array, n ):
    temp = list(combinations( array, n ))
    dic_combinations = {}
    for i in range(len(temp)): dic_combinations[ temp[i] ] = 0
    return dic_combinations

### REALIZA LA EVALUACION DE AQUELLAS CANCIONES O COMBINACIONES DE PLAYLIST QUE PASAN EL MIN_SUPPORT
def pass_min_support( dic, min_support ):
    min_frecuency = []
    for key, value in dic.items():
        if (value / 10000) >= min_support:min_frecuency.append( key )            
    return min_frecuency

### REALIZA LAS COMBINACIONES DE CANCIONES PARA UN SIGUIENTE NIVEL
def combinations_to_next_level( combinaciones_pass_support, nivel ):

    # RECUPERAMOS LOS N-1 ELEMENTOS DE CADA COMBINACION
    aux_primeros=[]
    for i in combinaciones_pass_support: aux_primeros.append( i[0] )

    # REALIZAMOS UN CONTEO PARA TRABAJAR CON AQUELLAS COMBINACIONES QUE TENGAN MAS DE UNA APARICION
    aUnique = np.array(aux_primeros)
    unique, counts = np.unique(aUnique, return_counts=True)
    auxi = dict(zip(unique, counts))

    aux_primeros_fijos = []
    combinatorias = {}
    for key, value in auxi.items(): # CONTEO DE APARICIONES DE LOS N-1 ELEMENTOS
        adicionales = []
        # SI TIENE MAS DE UNA APARICION SE TRABAJO CON SUS CONSECUENTES
        if value > 1:
            aux_primeros_fijos.append( key )
            for auxiliar in combinaciones_pass_support: # BUSQUEDA DE ADICIONALES EN DICCIONARIO 
                if key == auxiliar[0]:
                    adicionales.append(auxiliar[1])
            # SE REALIZA LAS COMBINACIONES CON LOS CONSECUENTES DE LAS CANCIONES
            adicionales = [key] + adicionales
            combinaciones = list(combinations(adicionales, nivel + 1))
            # AGREGAMOS LAS COMBINACIONES A UN DICCIONARIO
            for i in combinaciones:  
                if i[0] in aux_primeros_fijos:
                    combinatorias[ i ] = 0

    return combinatorias

### GENERA LAS REGLAS A PARTIR DE ITEMSETS FRECUENTES
def combinations_rules(items):
    dic = []
    # RECORREMOS LOS ITEMSETS FRECUENTES
    for i in range(len(items)):
        # REALIZAMOS LAS COMBINACIONES, DIVIDIENDO EL ITEM EN DOS SECCIONES
        tamanio = len(items[i])
        adicionales = list( items[i] )
        combinaciones = list(combinations(adicionales, tamanio - 1))

        for combinacion in combinaciones:
            inicio = set(items[i]) - set(combinacion)
            fin = combinacion
            # AGREGAMOS A UN ARREGLO, SIN REPETICIONES
            if [tuple(inicio), tuple(fin) ] not in dic:
                dic.append([tuple(inicio), tuple(fin) ])

            if [tuple(fin), tuple(inicio) ] not in dic:
                dic.append([tuple(fin), tuple(inicio) ])
    return dic

# FUNCION GET_FREQUENT_ITEMSETS

In [49]:
def get_frequent_itemsets(playlists, min_support):

    # FRECUENCIA DE APARICION DE CADA CANCION
    frecuency_of_songs = frecuency_each_song(playlists)

    # FILTRAR AQUELLAS CANCIONES QUE PASEN EL SOPORTE MINIMO
    songs_pass_support = pass_min_support( frecuency_of_songs, min_support )
    
    itemsets = []      # ARREGLO QUE GUARDA LOS ITEMSETS
    bandera = True     # BANDERA QUE NOS INDICARA SI SE SEGUIRA AVANZANDO EN NIVELES
    nivel = 1          # VARIABLE QUE INDICA EL NIVEL ACTUAL
    combinaciones = {} # DICCIONARIO QUE GUARDA LAS COMBINACIONES

    while ( bandera ):
        nivel += 1

        # HALLA LAS COMBINACIONES PARA EL RESPECTIVO NIVEL
        if nivel == 2: combinaciones = create_combinations( songs_pass_support, nivel )

        # HALLA LA FRECUENCIA CON LA QUE APARECE CADA COMBINACION EN CADA PLAYLIST
        frecuencia_combinaciones = frecuency_each_combination( spotify, combinaciones, nivel )

        # FILTRAR AQUELLAS COMBINACIONES QUE PASEN EL SOPORTE MINIMO
        combinaciones_pass_support = pass_min_support( frecuencia_combinaciones, min_support )

        # AGREGAR COMBINACIONES A 'ITEMSETS'
        for combinacion in combinaciones_pass_support: itemsets.append( combinacion )

        # ENCONTRAR LAS NUEVAS COMBINACIONES PARA EL SIGUIENTE NIVEL
        combinaciones = combinations_to_next_level( combinaciones_pass_support, nivel )

        # SI EL TAMAÑO DE COMBINACIONES QUE PASEN EL SOPORTE MINIMO ES '1', CORTAMOS EL ALGORITMO
        if (len( combinaciones_pass_support ) <= 1) : bandera = False

    return itemsets

# FUNCION GENERATE_ASSOCIATION_RULES

In [63]:
def generate_association_rules(frequent_itemsets, confidence, lift):

    # ENCONTRAMOS LAS REGLAS DE ASOCIACION, PARA LOS ITEMSETS QUE PASEN EL UMBRAL
    rules = combinations_rules(frequent_itemsets)

    count_x_y = [0] * len(rules) # CONTEO DE TRANSACCIONES (X U Y)
    count_x   = [0] * len(rules) # CONTEO DE TRANSACCIONES (X)
    count_y   = [0] * len(rules) # CONTEO DE TRANSACCIONES (Y)

    soporte   = [0] * len(rules) # CONTEO DE SOPORTE DE LOS ITEMS
    confianza = [0] * len(rules) # CONTEO DE CONFIANZA DE LOS ITEMS
    sustento  = [0] * len(rules) # CONTEO DE LIFT DE LOS ITEMS

    for i in range( len(spotify) ):   #len(arr)
    # RECORRER DICCIONARIO DE PLAYLISTS PARA CONTAR LAS TRANSACCIONES
        for j in range(len(rules)):
            
            x_y = list( rules[j][0] + rules[j][1] )
            x = list( rules[j][0] )
            y = list( rules[j][1] )

            # VERIFICAR SI COMBINACION SE ENCUENTRA EN PLAYLIST
            if( set(x_y).issubset(set( spotify[i] ))):  
                count_x_y[j] += 1
            if( set(x).issubset(set( spotify[i] ))):  
                count_x[j] += 1
            if( set(y).issubset(set( spotify[i] ))):  
                count_y[j] += 1
    # RECORRER DICCIONARIO DE RULES PARA CONTAR ATRIBUTOS
    for j in range(len(rules)):
        soporte[j] = (count_x_y[j] / len(spotify))
        confianza[j] = (count_x_y[j] / count_x[j])
        sustento[j] = confianza[j] / (count_y[j] / len(spotify))
    # IMPRIMIR LOS DATOS NECESARIOS EN CASO PASEN EL UMBRAL DE CONFIDENCE Y LIFT
    cont = 0
    for j in range(len(rules)):
        if (confianza[j] >= confidence) and (sustento[j] >= lift):
            print(cont+1)
            print("Regla: " , list(rules[j][0]) , " -> " , list(rules[j][1]))
            print("Soporte: " , str(soporte[j]*100))
            print("Confianza: " , str(confianza[j]*100))
            print("Sustento: " , str(sustento[j]))
            print("=====================================")
            cont += 1
    print(cont)


# MAIN

In [51]:
# OBTENEMOS LOS ITEMSETS MAS FRECUENTES

itemsets = get_frequent_itemsets(spotify, 0.015)

print( itemsets )

[('Closer', 'Let Me Love You'), ('goosebumps', 'XO TOUR Llif3'), ('goosebumps', 'HUMBLE.'), ('goosebumps', 'Congratulations'), ('XO TOUR Llif3', 'HUMBLE.'), ('XO TOUR Llif3', 'Mask Off'), ('XO TOUR Llif3', 'Congratulations'), ('iSpy (feat. Lil Yachty)', 'HUMBLE.'), ('iSpy (feat. Lil Yachty)', 'Bad and Boujee (feat. Lil Uzi Vert)'), ('iSpy (feat. Lil Yachty)', 'Congratulations'), ('DNA.', 'HUMBLE.'), ('HUMBLE.', 'Mask Off'), ('HUMBLE.', 'Bad and Boujee (feat. Lil Uzi Vert)'), ('HUMBLE.', 'Bounce Back'), ('HUMBLE.', 'Congratulations'), ('Mask Off', 'Bad and Boujee (feat. Lil Uzi Vert)'), ('Mask Off', 'Congratulations'), ('Broccoli (feat. Lil Yachty)', 'Bad and Boujee (feat. Lil Uzi Vert)'), ('Broccoli (feat. Lil Yachty)', 'Caroline'), ('Broccoli (feat. Lil Yachty)', 'No Problem (feat. Lil Wayne & 2 Chainz)'), ('Bad and Boujee (feat. Lil Uzi Vert)', 'Bounce Back'), ('Bad and Boujee (feat. Lil Uzi Vert)', 'Caroline')]


In [65]:
# GENERAMOS LAS REGLAS DE ASOCIACION

generate_association_rules(itemsets, confidence = 0.50, lift = 12)

1
Regla:  ['goosebumps']  ->  ['Congratulations']
Soporte:  1.55
Confianza:  50.324675324675326
Sustento:  12.487512487512488
2
Regla:  ['XO TOUR Llif3']  ->  ['HUMBLE.']
Soporte:  2.04
Confianza:  63.1578947368421
Sustento:  13.582342954159591
3
Regla:  ['Mask Off']  ->  ['XO TOUR Llif3']
Soporte:  1.63
Confianza:  51.58227848101266
Sustento:  15.969745659756239
4
Regla:  ['XO TOUR Llif3']  ->  ['Mask Off']
Soporte:  1.63
Confianza:  50.464396284829725
Sustento:  15.96974565975624
5
Regla:  ['XO TOUR Llif3']  ->  ['Congratulations']
Soporte:  1.79
Confianza:  55.417956656346746
Sustento:  13.751354009019042
6
Regla:  ['DNA.']  ->  ['HUMBLE.']
Soporte:  1.9
Confianza:  82.25108225108225
Sustento:  17.688404785178978
7
Regla:  ['Mask Off']  ->  ['HUMBLE.']
Soporte:  2.04
Confianza:  64.55696202531645
Sustento:  13.883217639853001
8
Regla:  ['Mask Off']  ->  ['Congratulations']
Soporte:  1.6199999999999999
Confianza:  51.26582278481012
Sustento:  12.721047837421866
9
Regla:  ['Bounce Bac

# EXPLICANDO REGLAS OBTENIDAS

### Se selecciono las siguientes reglas, ya que tienen el mejor Lift

REGLA N°1
* Regla    :  'Mask Off'  ->  'HUMBLE.'
* Soporte  :  2.04
* Confianza:  64.55696202531645
* Sustento :  13.883217639853001

----------------------------------------------------------------------------
REGLA N°2
* Regla    :  'XO TOUR Llif3'  ->  'Mask Off'
* Soporte  :  1.63
* Confianza:  50.464396284829725
* Sustento :  15.96974565975624

----------------------------------------------------------------------------

REGLA N°3
* Regla    :  'DNA.'  ->  'HUMBLE.'
* Soporte  :  1.9
* Confianza:  82.25108225108225
* Sustento :  17.688404785178978

----------------------------------------------------------------------------

REGLA N°4
* Regla    :  Bounce Back  ->  Bad and Boujee (feat. Lil Uzi Vert)
* Soporte  :  1.69
* Confianza:  56.9023569023569
* Sustento :  16.493436783291852

----------------------------------------------------------------------------


| CANCIÓN | ARTISTA | GÉNERO |
| --- | --- | --- |
| Mask Off | Future | Hip-hop/rap |
| HUMBLE | Kendrick Lamar | Hip-hop/rap |
| XO TOUR Llif3 | Lil Uzi Vert | Hip-hop/rap |
| DNA | Kendrick Lamar | Hip-hop/trap |
| Bounce Back | Big Sean | Hip-hop/rap |
| Bad and Boujee (feat. Lil Uzi Vert) | Migos | Hip-hop/trap |

### De estas canciones se concluye lo siguiente

- HUMBLE y DNA pertenecen al artista Kendrick Lamar, eso podría explicar porque tienen el lift y confianza mas alto.
- Todas las canciones pertenecen al genero Hip-hop, con pequeñas variaciones de trap y rap.
- Los artistas de MASK OFF y HUMBLE tienen canciones juntos, eso podría explicar su alto nivel de confianza.